# 🚀 Feature Extraction for Smishing Detection

Notebook này thực hiện trích xuất đặc trưng (feature extraction) cho toàn bộ dataset SMS nhằm phát hiện tin nhắn Smishing.

## Các bước thực hiện:
1. **Load Dataset**: Đọc dữ liệu từ `data/raw/dataset.csv`
2. **Feature Extraction**: Trích xuất 31 features từ mỗi tin nhắn
3. **Feature Selection**: Chọn 5 features quan trọng nhất (theo paper)
4. **Save Results**: Lưu kết quả vào `data/processed/`

**Tham khảo**: DSmishSMS Paper - A System to Detect Smishing SMS

## 1. Import Libraries và Setup


In [2]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Thêm thư mục src vào path để import module
sys.path.append('./src')

# Import module features
from features import (
    extract_all_features,
    extract_features_from_dataframe,
    get_selected_features_df,
    get_top_5_features
)   

print("✅ Import thành công!")
print(f"📅 Thời gian: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ Import thành công!
📅 Thời gian: 2025-12-09 17:59:30


## 2. Load Dataset


In [3]:
# Đọc dataset
dataset_path = 'data/raw/dataset.csv'

print(f"📂 Đang đọc dataset từ: {dataset_path}")
df = pd.read_csv(dataset_path)

print(f"✅ Đã load dataset thành công!")
print(f"📊 Số lượng tin nhắn: {len(df)}")
print(f"📋 Các cột: {df.columns.tolist()}")
print(f"\n📈 Phân bố nhãn:")
print(df['label'].value_counts())
print(f"\n🔍 Preview 3 dòng đầu:")
df.head(3)


📂 Đang đọc dataset từ: data/raw/dataset.csv
✅ Đã load dataset thành công!
📊 Số lượng tin nhắn: 2603
📋 Các cột: ['content', 'label', 'has_url', 'has_phone_number', 'sender_type']

📈 Phân bố nhãn:
label
0    2319
1     284
Name: count, dtype: int64

🔍 Preview 3 dòng đầu:


,content,label,has_url,has_phone_number,sender_type
0,[TRUNG TÂM PHÒNG CHỐNG GIAN LẬN NGÂN HÀNG] Ông...,1,0,0,personal_number
1,[TB] Tin ich Loi nham thoai cua Viettel: Quy k...,1,0,1,shortcode
2,Western Union TB: Vietcombank: 0071000986547. ...,1,1,0,personal_number


## 3. Feature Extraction

Trích xuất **31 features** từ mỗi tin nhắn SMS, bao gồm:
- **URL Features** (4): has_url, num_urls, has_suspicious_domain, url_length_avg
- **Phone Features** (4): has_phone, num_phones, has_personal_phone, has_hotline
- **Text Features** (8): message_length, num_words, num_digits, digit_ratio, etc.
- **Keyword Features** (11): financial, urgency, action, reward, impersonation keywords
- **Sender Features** (4): is_brandname, is_shortcode, is_personal_number, is_unknown


In [4]:
print("🔄 Bắt đầu trích xuất features...")
print(f"⏳ Ước tính thời gian: ~{len(df) * 0.1:.0f} giây ({len(df)} tin nhắn)")
print("-" * 80)

# Trích xuất features cho toàn bộ dataset
start_time = datetime.now()

df_with_features = extract_features_from_dataframe(
    df, 
    content_col='content', 
    sender_col='sender_type'
)

end_time = datetime.now()
duration = (end_time - start_time).total_seconds()

print("-" * 80)
print(f"✅ Hoàn thành feature extraction!")
print(f"⏱️  Thời gian thực thi: {duration:.2f} giây ({duration/len(df):.3f}s/tin nhắn)")
print(f"📊 Số features mới được tạo: {len(df_with_features.columns) - len(df.columns)}")
print(f"📋 Tổng số cột: {len(df_with_features.columns)}")
print(f"\n🔍 Danh sách features mới:")
new_cols = [col for col in df_with_features.columns if col not in df.columns]
for i, col in enumerate(new_cols, 1):
    print(f"   {i:2d}. {col}")


🔄 Bắt đầu trích xuất features...
⏳ Ước tính thời gian: ~260 giây (2603 tin nhắn)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
✅ Hoàn thành feature extraction!
⏱️  Thời gian thực thi: 1.72 giây (0.001s/tin nhắn)
📊 Số features mới được tạo: 32
📋 Tổng số cột: 37

🔍 Danh sách features mới:
    1. num_urls
    2. has_suspicious_domain
    3. url_length_avg
    4. has_phone
    5. num_phones
    6. has_personal_phone
    7. has_hotline
    8. message_length
    9. num_words
   10. num_digits
   11. digit_ratio
   12. num_special_chars
   13. special_char_ratio
   14. num_uppercase
   15. uppercase_ratio
   16. has_mixed_language
   17. num_financial_keywords
   18. num_urgency_keywords
   19. num_action_keywords
   20. num_reward_keywords
   21. num_impersonation_keywords
   22. has_financial_keywords
   23. has_urgency_keywords
   24. has_action_keywords
   25. has_reward_key

## 4. Data Analysis - Phân tích Features


In [5]:
# Thống kê mô tả cho các features
print("📊 THỐNG KÊ MÔ TẢ CHO CÁC FEATURES")
print("=" * 80)

# Chọn một số features quan trọng để xem
important_features = [
    'has_url', 'has_phone', 'num_financial_keywords', 
    'num_urgency_keywords', 'num_action_keywords',
    'message_length', 'num_words', 'digit_ratio',
    'is_brandname', 'is_personal_number'
]

print("\n📈 Thống kê cơ bản:")
print(df_with_features[important_features].describe().round(2))

print("\n" + "=" * 80)
print("🔍 PHÂN TÍCH THEO NHÃN (SMISHING vs NOT SMISHING)")
print("=" * 80)

for feature in ['has_url', 'has_phone', 'num_financial_keywords', 'num_urgency_keywords']:
    print(f"\n{feature}:")
    print(df_with_features.groupby('label')[feature].agg(['mean', 'sum', 'count']).round(3))


📊 THỐNG KÊ MÔ TẢ CHO CÁC FEATURES

📈 Thống kê cơ bản:
       has_url  has_url  has_phone  num_financial_keywords  \
count  2603.00  2603.00    2603.00                 2603.00   
mean      0.51     0.45       0.23                    0.50   
std       0.50     0.50       0.42                    1.04   
min       0.00     0.00       0.00                    0.00   
25%       0.00     0.00       0.00                    0.00   
50%       1.00     0.00       0.00                    0.00   
75%       1.00     1.00       0.00                    1.00   
max       1.00     1.00       1.00                    8.00   

       num_urgency_keywords  num_action_keywords  message_length  num_words  \
count               2603.00              2603.00         2603.00    2603.00   
mean                   0.64                 1.07          225.82      43.62   
std                    0.77                 1.41          107.70      22.12   
min                    0.00                 0.00            2.00       

## 5. Feature Selection - Chọn Top 5 Features

Dựa trên paper **DSmishSMS**, chúng ta chọn **5 features quan trọng nhất** để sử dụng cho mô hình phân loại.


In [6]:
# Lấy top 5 features
top_5_features = get_top_5_features()

print("🏆 TOP 5 FEATURES ĐƯỢC CHỌN:")
print("=" * 80)
for i, feature in enumerate(top_5_features, 1):
    print(f"   {i}. {feature}")

# Tạo DataFrame chỉ chứa top 5 features + label
df_top5 = get_selected_features_df(df_with_features)

print(f"\n📊 Shape của dataset với Top 5 features: {df_top5.shape}")
print(f"📋 Các cột: {df_top5.columns.tolist()}")
print("\n🔍 Preview 5 dòng đầu:")
df_top5.head()


🏆 TOP 5 FEATURES ĐƯỢC CHỌN:
   1. has_url
   2. has_phone
   3. num_financial_keywords
   4. num_urgency_keywords
   5. is_personal_number

📊 Shape của dataset với Top 5 features: (2603, 7)
📋 Các cột: ['label', 'has_url', 'has_url', 'has_phone', 'num_financial_keywords', 'num_urgency_keywords', 'is_personal_number']

🔍 Preview 5 dòng đầu:


,label,has_url,has_url,has_phone,num_financial_keywords,num_urgency_keywords,is_personal_number
0,1,0,0,0,7,2,1
1,1,0,0,1,8,2,0
2,1,1,1,0,3,2,1
3,1,1,0,0,3,0,1
4,1,0,0,0,0,1,1


## 6. Save Results - Lưu Kết Quả

Lưu 2 phiên bản dataset:
1. **features_full.csv**: Chứa tất cả 31 features
2. **features_top5.csv**: Chỉ chứa 5 features quan trọng nhất (để train model)


In [7]:
# Tạo thư mục processed nếu chưa có
processed_dir = 'data/processed'
os.makedirs(processed_dir, exist_ok=True)

# Đường dẫn file output
full_features_path = os.path.join(processed_dir, 'features_full.csv')
top5_features_path = os.path.join(processed_dir, 'features_top5.csv')

print("💾 Đang lưu kết quả...")
print("-" * 80)

# Lưu full features
df_with_features.to_csv(full_features_path, index=False, encoding='utf-8-sig')
print(f"✅ Đã lưu full features: {full_features_path}")
print(f"   📊 Shape: {df_with_features.shape}")
print(f"   💾 Kích thước: {os.path.getsize(full_features_path) / 1024:.2f} KB")

# Lưu top 5 features
df_top5.to_csv(top5_features_path, index=False, encoding='utf-8-sig')
print(f"\n✅ Đã lưu top 5 features: {top5_features_path}")
print(f"   📊 Shape: {df_top5.shape}")
print(f"   💾 Kích thước: {os.path.getsize(top5_features_path) / 1024:.2f} KB")

print("\n" + "=" * 80)
print("🎉 HOÀN THÀNH FEATURE EXTRACTION!")
print("=" * 80)
print("📂 Các file đã tạo:")
print(f"   1. {full_features_path}")
print(f"   2. {top5_features_path}")
print("\n🚀 Bước tiếp theo: Train model với features_top5.csv")


💾 Đang lưu kết quả...
--------------------------------------------------------------------------------
✅ Đã lưu full features: data/processed\features_full.csv
   📊 Shape: (2603, 37)
   💾 Kích thước: 1039.82 KB

✅ Đã lưu top 5 features: data/processed\features_top5.csv
   📊 Shape: (2603, 7)
   💾 Kích thước: 38.23 KB

🎉 HOÀN THÀNH FEATURE EXTRACTION!
📂 Các file đã tạo:
   1. data/processed\features_full.csv
   2. data/processed\features_top5.csv

🚀 Bước tiếp theo: Train model với features_top5.csv


## 7. Sample Results - Xem Ví Dụ Kết Quả

Xem một số ví dụ cụ thể về features được trích xuất từ tin nhắn Smishing và Not Smishing


In [8]:
print("📝 VÍ DỤ SMISHING MESSAGES (label=1)")
print("=" * 80)

# Lấy 2 ví dụ smishing
smishing_samples = df_with_features[df_with_features['label'] == 1].head(2)

for idx, row in smishing_samples.iterrows():
    print(f"\n🚨 Tin nhắn #{idx}")
    print(f"Content: {row['content'][:150]}...")
    print(f"Sender: {row['sender_type']}")
    print(f"\nTop 5 Features:")
    for feat in get_top_5_features():
        print(f"   • {feat:25s}: {row[feat]}")
    print("-" * 80)

print("\n\n📝 VÍ DỤ HAM/LEGITIMATE MESSAGES (label=0)")
print("=" * 80)

# Lấy 2 ví dụ ham
ham_samples = df_with_features[df_with_features['label'] == 0].head(2)

for idx, row in ham_samples.iterrows():
    print(f"\n✅ Tin nhắn #{idx}")
    print(f"Content: {row['content'][:150]}...")
    print(f"Sender: {row['sender_type']}")
    print(f"\nTop 5 Features:")
    for feat in get_top_5_features():
        print(f"   • {feat:25s}: {row[feat]}")
    print("-" * 80)


📝 VÍ DỤ SMISHING MESSAGES (label=1)

🚨 Tin nhắn #0
Content: [TRUNG TÂM PHÒNG CHỐNG GIAN LẬN NGÂN HÀNG] Ông/Bà Nguyễn Văn Minh Trước 17h ngày hôm nay không thanh toán dứt điểm số tiền quá hạn 910.046 VNĐ. Chúng ...
Sender: personal_number

Top 5 Features:
   • has_url                  : has_url    0
has_url    0
Name: 0, dtype: object
   • has_phone                : 0
   • num_financial_keywords   : 7
   • num_urgency_keywords     : 2
   • is_personal_number       : 1
--------------------------------------------------------------------------------

🚨 Tin nhắn #1
Content: [TB] Tin ich Loi nham thoai cua Viettel: Quy khach co loi nhan tu TB 0848836182 vao luc 08:09 27/03/2025. Voi noi dung: kính chào quý khách đây là cuộ...
Sender: shortcode

Top 5 Features:
   • has_url                  : has_url    0
has_url    0
Name: 1, dtype: object
   • has_phone                : 1
   • num_financial_keywords   : 8
   • num_urgency_keywords     : 2
   • is_personal_number       : 0
-----------------

## 📊 Summary - Tổng Kết

**Feature Extraction hoàn thành!** 🎉

### ✅ Công việc đã hoàn thành:
- ✔️ Load dataset: 2,618 tin nhắn SMS
- ✔️ Trích xuất 31 features từ mỗi tin nhắn
- ✔️ Chọn Top 5 features quan trọng nhất
- ✔️ Lưu 2 file CSV: `features_full.csv` và `features_top5.csv`

### 🏆 Top 5 Features được chọn:
1. `has_url` - Có URL không?
2. `has_phone` - Có số điện thoại không?
3. `num_financial_keywords` - Số lượng từ khóa tài chính
4. `num_urgency_keywords` - Số lượng từ khóa khẩn cấp
5. `is_personal_number` - Gửi từ SĐT cá nhân?

### 🚀 Bước tiếp theo:
1. **Train Model**: Sử dụng `features_top5.csv` để train các mô hình ML
2. **Domain Checking Phase**: Implement kiểm tra độ tin cậy của URL
3. **Model Evaluation**: Đánh giá accuracy, precision, recall, F1-score
4. **Compare với Paper**: So sánh kết quả với DSmishSMS paper (97.93% accuracy)

### 📂 Output Files:
- `data/processed/features_full.csv` - Tất cả 31 features
- `data/processed/features_top5.csv` - 5 features chính để train model
